In [ ]:
import torch
from torch import nn
from models import CustomResNet18, device
from dataset_utils import get_dataloaders, pacs_df

In [ ]:
resnet18 = CustomResNet18()
resnet18.to(device)

train_loader, test_loader = get_dataloaders(pacs_df, 0.2)

resnet18.train()

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(resnet18.parameters(), lr=0.001)

num_epochs = 10

for epoch in range(num_epochs):
    running_loss = 0.0
    for i, (features, labels) in enumerate(train_loader):
        features, labels = features.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = resnet18(features)

        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if (i + 1) % 10 == 0:
            print(
                f"Epoch [{epoch + 1}/{num_epochs}], Step [{i + 1}/{len(train_loader)}], Loss: {running_loss / 10:.4f}"
            )
            running_loss = 0.0

print("Training complete")